# Symmetric Encryption

Encryption is the process of transforming a message, usually called cyphered message, in such a way that only the intended parties can reverse it. The process of reversing encryption is called decryption. This implies that, as opposed to hashes, encryption is **reversible**.

There are two main types of encryption, symmetric and asymmetric, this chapter will cover symmetric encryption.

In symmetric encryption, the message to be sent is encrypted using a single **secret** password, also called key. Anyone with that secret key and decrypt the message and see the original content. 

This type of encryption is fast, reliable and extremely secure, however it has a major disadvantages, the **secret** key must be shared between the intended parties and the whole encryption procedure will be as secure as the security used for that key exchange. That is why usually other mechanisms such as asymmetric encryption are used only for the key-exchange.

## Encryption != Encoding

Some times this terms are confused, **encryption** as mentioned is reversible but only by the intended parties whereas **encoding** is simply a way to transform a message in a **non-secure** way, mainly as means of compression or to transform a message into a more suitable format. 

One of the most common encodings used is Base64, which can be used to send binary files (PDFs, images, etc.) as plain text in HTTP Requests. Working with Base64 is a topic outside of this guide, however some introductory material is provided in [one of the appendixes](92_Base64.ipynb). 

## Authenticated vs Non-Authenticated Symmetric Encryption

A futher categorization of symmetric encryption algorithms is between those with authentication features and those without. Both provide a way to securely transmit a meesage, however, [authenticated Encryption](https://en.wikipedia.org/wiki/Authenticated_encryption) also allows, among other things, to verify whether the ciphered message has been modified.

Modifying the ciphered message in a non-authenticated scheme will produce a nonsensical output whereas in authenticated encryption it will throw an error.

## The Cryptography Library

Python does not include any symmetric encryption features in its standard library, therefore, to use it one should install a third party tool. It is of great importance that the library to be used has some characteristics to ensure real security:

- It should be built only on the standard library, this will ensure full compatibility and avoid dependency-related problems.
- It should be open source, that way the code could be review and everyone knows exactly how it works.
- It should be time-proved, new libraries tend to have bugs or poor implementations, libraries with some time online are usually prefered.
- It should be used by trustworthy users, if it is an industry standard it means many companies and individuals trust it.

The Python Cryptographic Authority, or pyca for short, is a non-official group of users who developed cryptographic libraries for this very purpose. It supports symmetric (authenticated and non-authenticated) and asymmetric encryption.

The most common and used is homonomous to the group name: `cryptography`

It can be install via `pip` with:

```
pip install cryptography
```

In [1]:
import secrets
import cryptography

## Symmetric Encryption without Authentication

One of the most popular algorithms used is the [Advanced Encryption Standard (AES)](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard). It works in combination with a [Mode of Operation](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation), this modes change the way in which the cyphered message is generated.

Depending on the mode, the result could be authenticated or non-authenticated. The available (in the cryptography module) modes for non-authenticated encryption are [CBC](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_block_chaining_(CBC)), [CFB](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_feedback_(CFB)), [OFB](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Output_feedback_(OFB)), [CTR](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Counter_(CTR)). For more information see [this resource](https://www.highgo.ca/2019/08/08/the-difference-in-five-modes-in-the-aes-encryption-algorithm/).

Usually the name of the algorithm is called **AES-MODE** in this chapter for non-authenticated encryption **AES-CTR** will be used.

AES is one of the symmetric encryption algorithms available in the PyCA `cryptography` library. The particular objects used here are part of the `hazmat` package, `hazmat` stands for "Hazardous Materials" and quoting from their [site](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/):

> This is a “Hazardous Materials” module. You should ONLY use it if you’re 100% absolutely sure that you know what you’re doing because this module is full of land mines, dragons, and dinosaurs with laser guns.

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Encrypting

In [3]:
message = b"Hello World!"

key = secrets.token_bytes(32)
initialization_vector = secrets.token_bytes(16)

algorithm = algorithms.AES(key)
mode = modes.CTR(initialization_vector)

cipher = Cipher(algorithm, mode)
encryptor = cipher.encryptor()

message_encrypted = encryptor.update(message) + encryptor.finalize()

print(f"Secret Key: {key.hex()}")
print(f"Public Initialization Vector: {initialization_vector.hex()}")
print(f"Encrypted Message: {message_encrypted.hex()}")

Secret Key: e50a173fd3792c619b5c5863baf394df2620114babf48bc505b52a0c4974991a
Public Initialization Vector: 0ec5fc0c419d0e8b1a8efc77f00acaed
Encrypted Message: f8fc6e27702f6de3e4c5d540


### Decrypting

#### Wrong Key

In [4]:
guess_password = secrets.token_bytes(32)

algorithm = algorithms.AES(guess_password)
mode = modes.CTR(initialization_vector)

cipher = Cipher(algorithm, mode)

decryptor = cipher.decryptor()
message_decrypted = decryptor.update(message_encrypted) + decryptor.finalize()

print(f"Decrypted Message: {message_decrypted}")

Decrypted Message: b'\xfc\xbeJ\xd2lR\xaf\xddD\x1e%\xe0'


#### Right Key

In [5]:
algorithm = algorithms.AES(key)
mode = modes.CTR(initialization_vector)

cipher = Cipher(algorithm, mode)

decryptor = cipher.decryptor()
message_decrypted = decryptor.update(message_encrypted) + decryptor.finalize()
print(f"Decrypted Message: {message_decrypted}")

Decrypted Message: b'Hello World!'


## Symmetric Encryption with Authentication

The same AES algorithm can be combined with modes that results in a authenticated encryption. Such modes are [GCM](https://en.wikipedia.org/wiki/Galois/Counter_Mode), [OCB3](https://en.wikipedia.org/wiki/OCB_mode) and [CCM](https://en.wikipedia.org/wiki/CCM_mode).

Even though they provide more features that the non-authenticated modes, they are sensitive to nonce re-use, having known vulnerabilities if the same nonce is used twice. There is a new generation of modes called **Synthetic Initialization Vector (SIV)** which mitigates this risks, producing modes like [AES-GCM-SIV](https://en.wikipedia.org/wiki/AES-GCM-SIV) and so on, this new generation has no time-proved implementation neither in Python nor in other programming language at the time of this writting.

In [6]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

### Encrypting

In [7]:
message = b"Hello World!"

key = secrets.token_bytes(32) # 16, 24, or 32
nonce = secrets.token_bytes(24)

ciphered_message = AESGCM(key).encrypt(nonce, message, None)

encrypted_message = f"{nonce.hex()}:{ciphered_message.hex()}"

print(f"Secret Key: {key.hex()}")
print(f"Public Nonce: {nonce.hex()}")
print(f"Encrypted Message: {encrypted_message}")

Secret Key: 4077459071052930a654b3ac2d74094433a955fcf6bdef87c40f5e5b4d826c1d
Public Nonce: 515406389dcb4e51d795eda30658391d2f0c3f78e5a5549f
Encrypted Message: 515406389dcb4e51d795eda30658391d2f0c3f78e5a5549f:06636d8fa9fe7f671c30eb4bb950594faab6f3cbec4314743c3a86f9


### Decrypting

In [8]:
def verify(password, nonce, message):
    try:
        decrypted_message = AESGCM(password).decrypt(nonce, message, None)
        return f"Decrypted Message: {message_decrypted}"
    except cryptography.exceptions.InvalidTag:
        return "Verification Failed - Either the message has been altered or the nonce or key are incorrect"

#### Wrong Key

In [9]:
guess_password = bytes.fromhex("1329f363a87306c33952a7cbfc0ebf8105126764d1c72c511031a5b028110cf9")

nonce, ciphered_message = encrypted_message.split(":")
nonce_bytes = bytes.fromhex(nonce)
ciphered_message_bytes = bytes.fromhex(ciphered_message)

print(verify(guess_password, nonce_bytes, ciphered_message_bytes))

Verification Failed - Either the message has been altered or the nonce or key are incorrect


#### Right Key

In [10]:
nonce, ciphered_message = encrypted_message.split(":")
nonce_bytes = bytes.fromhex(nonce)
ciphered_message_bytes = bytes.fromhex(ciphered_message)

print(verify(key, nonce_bytes, ciphered_message_bytes))

Decrypted Message: b'Hello World!'


## The Fernet Method

The [Fernet](https://github.com/fernet/spec/blob/master/Spec.md) method is an opinionated way of using AES and HMAC (see next chapter) to provide authenticated encryption. It is meant for easy use but it requires that the recipient also has a library compatible with it to effectively use it.

It is not nearly as commonly used as the other methods described in this chapter but it is much more practical for simple use cases.

At the time of this writing there are implementations for: 

- [Go (official)](https://github.com/fernet/fernet-go)
- [Ruby (official)](https://github.com/fernet/fernet-rb)
- [Erlang (official)](https://github.com/fernet/fernet-erl)
- [Python](https://cryptography.io/en/latest/fernet/) (i.e. using cryptography)
- [Javascript (Node)](https://www.npmjs.com/package/fernet).

This method also allows the usage of **passwords (human-readable)** instead of random bytes as keys, see [this example](https://cryptography.io/en/latest/fernet/#using-passwords-with-fernet) for more information.

The Fernet algorithm is not part of the `hazmat` package and then it is meant to be use as a developer friendly approach to symmetric encryption.

In [11]:
from cryptography.fernet import Fernet

### Encrypting

In [12]:
message = b"Hello World!"

key = Fernet.generate_key()

cipher = Fernet(key)

message_encrypted = cipher.encrypt(message)

print(f"Secret Key: {key.hex()}")
print(f"Encrypted Message: {message_encrypted.hex()}")

Secret Key: 6c71775159453056557278522d66537735303739593365506b7669475a586b546c426f6c45464444725f633d
Encrypted Message: 674141414141426878487732635562434c6a69726946596b627462526867577177767a364c5f57432d6e793938433468695a72716344435861587756676d5044455955306a64312d5a43445856306b6d5f626d32313951714642776c65305a5479673d3d


### Decrypting

#### Wrong Key

In [13]:
guess_password = bytes.fromhex("785754746c6136366d4f467431395543336c46444c71692d3249792d5a365374386665566f64726f4639303d")

cipher = Fernet(guess_password)

try:
    message_decrypted = cipher.decrypt(message_encrypted)
    print(f"Encrypted Message: {message_decrypted}")
except cryptography.fernet.InvalidToken:
    print("Token Invalid")

Token Invalid


#### Right Key

In [14]:
cipher = Fernet(key)

try:
    message_decrypted = cipher.decrypt(message_encrypted)
    print(f"Encrypted Message: {message_decrypted}")
except cryptography.fernet.InvalidToken:
    print("Token Invalid")

Encrypted Message: b'Hello World!'


### Conclusion

Symmetric encryption provides a way to hide the original information in a reversible way. When using authenticated algorithms, if the encrypted message is modified, when decrypted, the output will be have no sense. Authenticated algorithms can detect changes and throw errors. There are many implementations of symmetric algorithms, many are low-level like AES but are more cross-platform whereas other such as Fernet have a more user-friendly approach but are only avaiable in certain programming languages.